# 1 初始化

## 1.1 导入包、超参数定义

In [2]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from tqdm import tqdm

import logging
log_fmt = "[%(asctime)s] %(levelname)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"


import gzip
import pickle
base_path = '/home/zengrui/datasets/190829_Kanshan_zjfx'

def save_param(param, param_name):
    with open(f'{base_path}/temp/{param_name}.txt', 'wb') as f:
        pickle.dump(param, f)
        
def read_param(param_name):
    with open(f'{base_path}/temp/{param_name}.txt', 'rb') as f:
        return pickle.load(f)

def gload(file_name):
    with gzip.GzipFile(f'{base_path}/temp/{file_name}.gzip', 'rb') as f:
        return pickle.load(f)

def gdump(obj, file_name):
    with gzip.GzipFile(f'{base_path}/temp/{file_name}.gzip', 'wb') as f:
        pickle.dump(obj, f)

# 2 问题词偏好挖掘

## 2.1 加载数据

需要加载的数据：

* `question`: qid, title_t2, desc_t2
* `answer`: uid, qid
* `invite`: qid, uid

Load `invite`, `invite_evaluate`.

> `dtype`: obj->int, speed up **50x**.

In [10]:
train = pd.read_csv(f'{base_path}/invite_info_0926.txt', sep='\t', header=None)
train.columns = ['qid', 'uid', 'dt', 'label']
test = pd.read_csv(f'{base_path}/invite_info_evaluate_1_0926.txt', sep='\t', header=None)
test.columns = ['qid', 'uid', 'dt']

def extract_day(s):
    return s.apply(lambda x: int(x.split('-')[0][1:]))

train['day'] = extract_day(train['dt'])
train = train[train['day'] > 3860]

del train['dt'], train['label'], train['day'], test['dt']

# 优化id为int
train['qid'] = pd.to_numeric(train['qid'].apply(lambda x: x[1:]))
train['uid'] = pd.to_numeric(train['uid'].apply(lambda x: x[1:]))
test['qid'] = pd.to_numeric(test['qid'].apply(lambda x: x[1:]))
test['uid'] = pd.to_numeric(test['uid'].apply(lambda x: x[1:]))

train_size, test_size = len(train), len(test)

logging.info('train %s, test %s', train.shape, test.shape)
logging.info('columns: %s', train.columns)
logging.info('dtypes: %s', train.dtypes)

[2019-12-14 18:36:00,484] INFO: train (2593669, 2), test (1141683, 2)
[2019-12-14 18:36:00,487] INFO: columns: Index(['qid', 'uid'], dtype='object')
[2019-12-14 18:36:00,489] INFO: dtypes: qid    int64
uid    int64
dtype: object


Load `question`.

In [4]:
ques = pd.read_csv(f'{base_path}/question_info_0926.txt', header=None, sep='\t')
ques.columns = ['qid', 'q_dt', 'title_t1', 'title_t2', 'desc_t1', 'desc_t2', 'topic']

for i in ['q_dt', 'title_t1', 'desc_t1', 'topic']:
    del ques[i]

# 优化id为int
ques['qid'] = pd.to_numeric(ques['qid'].apply(lambda x: x[1:]))
ques['title_t2'] = ques['title_t2'].astype(str)
ques['desc_t2'] = ques['desc_t2'].astype(str)

logging.info('ques: %s', ques.shape)
logging.info('columns: %s', ques.columns)
logging.info('dtypes: %s', ques.dtypes)

[2019-12-14 18:33:23,671] INFO: ques: (1829900, 3)
[2019-12-14 18:33:23,674] INFO: columns: Index(['qid', 'title_t2', 'desc_t2'], dtype='object')
[2019-12-14 18:33:23,675] INFO: dtypes: qid          int64
title_t2    object
desc_t2     object
dtype: object


Load `answer`.

In [5]:
ans = pd.read_csv(f'{base_path}/answer_info_0926.txt', header=None, sep='\t')
ans.columns = ['aid', 'qid', 'uid', 'ans_dt', 'ans_t1', 'ans_t2', 'is_good', 'is_rec', 'is_dest', 'has_img',
               'has_video', 'word_count', 'reci_cheer', 'reci_uncheer', 'reci_comment', 'reci_mark', 'reci_tks',
               'reci_xxx', 'reci_no_help', 'reci_dis']

for i in ['aid', 'ans_dt', 'ans_t1', 'ans_t2', 'is_good', 'is_rec', 'is_dest', 
          'has_img', 'has_video', 'word_count', 'reci_cheer', 'reci_uncheer', 
          'reci_comment', 'reci_mark', 'reci_tks', 'reci_xxx', 'reci_no_help', 
          'reci_dis']:
    del ans[i]

# 优化id为int
ans['qid'] = pd.to_numeric(ans['qid'].apply(lambda x: x[1:]))
ans['uid'] = pd.to_numeric(ans['uid'].apply(lambda x: x[1:]))

logging.info("ans: %s", ans.shape)
logging.info("columns: %s", ans.columns)
logging.info("dtypes: %s", ans.dtypes)

[2019-12-14 18:34:39,551] INFO: ans: (4513735, 2)
[2019-12-14 18:34:39,553] INFO: columns: Index(['qid', 'uid'], dtype='object')
[2019-12-14 18:34:39,555] INFO: dtypes: qid    int64
uid    int64
dtype: object


## 2.2 提取数据

In [11]:
# 合并 train & test -> data
data = pd.concat((train, test), axis=0, sort=True)
del train, test

len(set(data['uid']))

1050498

### 2.2.1 创建uhist表

创建 **uid - 个人历史回答问题的标题拼接, 个人历史回答的描述拼接** 表 `uhist` (user_history)。

In [12]:
uhist = pd.DataFrame()
uhist['uid'] = pd.Series(list(set(data['uid'])))
print(uhist.dtypes)

uid    int64
dtype: object


读取每个人历史回答词拼接。（注意：速度上，`联表 + groupby` 计算远比 `apply` 快。）

In [13]:
# 联表 uhist - ans - ques
uhist = uhist.merge(ans, on='uid', how='left')
uhist = uhist.merge(ques, on='qid', how='left')
del uhist['qid'], ans

uhist = uhist.fillna('')  # 化NaN为''

print(len(uhist))
print(uhist.columns)

4627925
Index(['uid', 'title_t2', 'desc_t2'], dtype='object')


In [14]:
# 统计每个人所有 ques 的字符串拼接
uhist = pd.DataFrame(
    uhist.groupby('uid')['title_t2'].apply(
        lambda x: ','.join(list(filter(lambda x: x != '-1', x)))
    )
).merge(
    pd.DataFrame(
        uhist.groupby('uid')['desc_t2'].apply(
            lambda x: ','.join(list(filter(lambda x: x != '-1', x)))
        )
    ), on='uid', how='left'
)
uhist

,title_t2,desc_t2
uid,,
2786,"W13983,W35802,W5618,W1201,W4199,W16462,W840,W2...","W894,W188,W126,W17765,W1761,W848,W5019,W2376,W..."
9838,,
15476,"W11155,W10203,W2139,W856,W51,W184,W6135,W1276,...","W705,W14650,W51,W4274,W123,W11155,W938,W121536..."
16689,"W31273,W33874,W1222,W29020,W81814,W38692,W1241...","W546,W233,W270,W79,W412,W466,W522,W105161,W200..."
24521,"W973,W659,W5104,W26102,W3931",W689
...,...,...
4294959078,"W1969,W12985,W43374,W827,W35689,W490312,W73280...","W19761,W370,W30329,W41726,W3865,W93971,W233,W1..."
4294960406,,
4294962025,"W11550,W1684,W822,W622,W1411,W126,W6675,W227,W...","W1684,W11550,W1157,W822,W622,W319,W448,W5771,W..."


In [15]:
# 赶快存储 uhist
gdump(uhist, 'uhist')

### 2.2.2 数据汇集到data表

In [16]:
# ques, uhist 数据汇集到 data
data = data.merge(ques, on='qid', how='left')
data = data.merge(uhist, on='uid', how='left')

data.columns = ['qid', 'uid', 'title_t2', 'desc_t2', 'hist_title_t2', 'hist_desc_t2']
logging.info(data.columns)

data['title_t2'].replace('-1', '', inplace=True)
data['desc_t2'].replace('-1', '', inplace=True)

del ques, uhist

[2019-12-14 18:43:37,607] INFO: Index(['qid', 'uid', 'title_t2', 'desc_t2', 'hist_title_t2', 'hist_desc_t2'], dtype='object')


In [17]:
data

,qid,uid,title_t2,desc_t2,hist_title_t2,hist_desc_t2
0,2166419046,401693808,"W11058,W272,W2202,W431,W951,W243,W3828,W3037,W263","W3828,W14870,W3037,W2450,W692,W1688,W53547,W24...",,
1,604029601,2317670257,"W31489,W9218,W3440,W243","W3046,W221,W3440,W730,W31489,W9218,W1278,W415","W272,W263,W4534,W7742,W973,W3688,W3973,W258,W6...","W272,W4534,W7742,W3738,W1277,W111,W2425,W41838..."
2,2443223942,3544409350,"W554,W28208,W396,W51",,,
3,795459266,2818659842,"W8705,W5085,W185",,"W886,W188,W19433,W16846,W105,W1222,W6481,W2652...","W638,W886,W894,W188,W11222,W105,W715,W3995,W29..."
4,110462128,848334644,"W9097,W32077,W5634","W9097,W9335,W23221,W788,W6778,W37109,W12685,W1...","W1891,W9097,W58494,W140823,W2486,W1276,W351,W9...","W1090,W5074,W1914,W1968,W2126,W3495,W6658,W152..."
...,...,...,...,...,...,...
3735347,1238703523,2010778235,"W4084,W188,W9690,W1234",,"W9473,W7817,W516,W554,W1131,W9473,W7817,W1043,...","W799,W7817,W152,W9473,W7817,W1072,W1925,W9473,..."
3735348,1074024036,3131383616,"W7437,W689,W71889,W374,W2023,W11487,W710,W7703",,"W2199,W41177,W5193,W689,W675,W254,W599,W16166,...",
3735349,3478846332,1872860897,"W973,W38340","W403,W2051,W38340","W700,W39193,W3320,W5280,W3490",
3735350,734170704,3574631517,"W1072,W11709,W7011,W15073,W1772,W5089","W22564,W41704,W1433,W799,W17575,W5745,W20330,W...","W76,W1251,W303,W97,W1748,W878,W854,W8934,W614,...","W13012,W19891,W22257,W854,W8934,W2646,W221,W50..."


## 2.3 挖掘数据

挖掘 ['title_t2', 'desc_t2'] 两者中的以下数据：

* a. ques的word中曾经出现过个种 `len(set(word) & set(hist_word))`
* b. ques中曾经word占ques所有word比例 `a / len(set(word))`
* c. ques中曾经word占所有曾经word比例 `a / len(set(hist_word))`
* d. ques的word中曾经出现过个次 `len([x for x in word if x in (set(word) & set(hist_word))])`
* e. ques的word中曾经出现过个次占所有曾经出现过个次比例 `d / len(hist_word)`

```python
a = len(set(coauthor) & set(history_coauthor))
b = a / len(set(coauthor))
c = a / len(set(history_coauthor))
d = len([x for x in coauthor if x in (set(coauthor) & set(history_coauthor))])
e = d / len(history_coauthor)
```

In [116]:
# 一列 apply 生成多列的方法
# def tmp(x):
#     return x, x*2, x*3

# a, b, c = zip(*data[:1000]['uid'].apply(tmp))

In [18]:
pbar = tqdm(total=len(data))


def foo(word, hist_word):
    '''输入 ques_word 和 hist_word, 输出 a, b, c, d, e'''
    pbar.update()
    if 0 in [len(word), len(hist_word)]:  # 排除空输入
        return 0, 0, 0, 0, 0
    sw, shw = set(word), set(hist_word)
    lsw, lshw = len(sw), len(shw)
    and_set = sw & shw
    a = len(and_set)
    b = a / lsw
    c = a / lshw
    d = len([x for x in word if x in and_set])
    e = d / len(hist_word)
    return a, b, c, d, e


data['a'], data['b'], data['c'], data['d'], data['e'] = zip(*data[['title_t2', 'hist_title_t2']].apply(
    lambda r: foo(r['title_t2'], r['hist_title_t2']), axis=1))
pbar.close()

pbar = tqdm(total=len(data))
data['aa'], data['bb'], data['cc'], data['dd'], data['ee'] = zip(*data[['desc_t2', 'hist_desc_t2']].apply(
    lambda r: foo(r['desc_t2'], r['hist_desc_t2']), axis=1))
pbar.close()

data


 21%|██▏       | 801808/3735352 [00:28<01:33, 31495.36it/s]


 44%|████▎     | 1632641/3735352 [00:54<01:04, 32669.31it/s]


 66%|██████▌   | 2466043/3735352 [01:20<00:38, 32899.42it/s]


 89%|████████▊ | 3310013/3735352 [01:46<00:13, 32461.21it/s]


100%|██████████| 3735352/3735352 [02:05<00:00, 29776.86it/s]

  9%|▉         | 350469/3735352 [00:13<02:01, 27785.47it/s]


...

 48%|████▊     | 1779979/3735352 [01:06<01:17, 25366.56it/s]


 67%|██████▋   | 2490313/3735352 [01:32<00:44, 28223.76it/s]


 85%|████████▌ | 3188712/3735352 [01:58<00:20, 26766.17it/s]


100%|██████████| 3735352/3735352 [02:24<00:00, 25888.12it/s]


,qid,uid,title_t2,desc_t2,hist_title_t2,hist_desc_t2,a,b,c,d,e,aa,bb,cc,dd,ee
0,2166419046,401693808,"W11058,W272,W2202,W431,W951,W243,W3828,W3037,W263","W3828,W14870,W3037,W2450,W692,W1688,W53547,W24...",,,0,0.0,0.000000,0,0.000000,0,0.0,0.0,0,0.000000
1,604029601,2317670257,"W31489,W9218,W3440,W243","W3046,W221,W3440,W730,W31489,W9218,W1278,W415","W272,W263,W4534,W7742,W973,W3688,W3973,W258,W6...","W272,W4534,W7742,W3738,W1277,W111,W2425,W41838...",9,1.0,0.750000,23,0.209091,12,1.0,1.0,45,0.173077
2,2443223942,3544409350,"W554,W28208,W396,W51",,,,0,0.0,0.000000,0,0.000000,0,0.0,0.0,0,0.000000
3,795459266,2818659842,"W8705,W5085,W185",,"W886,W188,W19433,W16846,W105,W1222,W6481,W2652...","W638,W886,W894,W188,W11222,W105,W715,W3995,W29...",7,1.0,0.583333,16,0.064257,0,0.0,0.0,0,0.000000
4,110462128,848334644,"W9097,W32077,W5634","W9097,W9335,W23221,W788,W6778,W37109,W12685,W1...","W1891,W9097,W58494,W140823,W2486,W1276,W351,W9...","W1090,W5074,W1914,W1968,W2126,W3495,W6658,W152...",10,1.0,0.833333,18,0.008751,12,1.0,1.0,56,0.012267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3735347,1238703523,2010778235,"W4084,W188,W9690,W1234",,"W9473,W7817,W516,W554,W1131,W9473,W7817,W1043,...","W799,W7817,W152,W9473,W7817,W1072,W1925,W9473,...",10,1.0,0.833333,22,0.209524,0,0.0,0.0,0,0.000000
3735348,1074024036,3131383616,"W7437,W689,W71889,W374,W2023,W11487,W710,W7703",,"W2199,W41177,W5193,W689,W675,W254,W599,W16166,...",,11,1.0,0.916667,46,0.353846,0,0.0,0.0,0,0.000000
3735349,3478846332,1872860897,"W973,W38340","W403,W2051,W38340","W700,W39193,W3320,W5280,W3490",,8,1.0,0.727273,11,0.379310,0,0.0,0.0,0,0.000000
3735350,734170704,3574631517,"W1072,W11709,W7011,W15073,W1772,W5089","W22564,W41704,W1433,W799,W17575,W5745,W20330,W...","W76,W1251,W303,W97,W1748,W878,W854,W8934,W614,...","W13012,W19891,W22257,W854,W8934,W2646,W221,W50...",10,1.0,0.833333,37,0.077895,12,1.0,1.0,80,0.022353


## 2.4 压缩数据

In [19]:
# 删除没用的列
# for i in ['title_t2', 'hist_title_t2', 'desc_t2', 'hist_desc_t2']:
#     del data[i]
    
t = data.dtypes
for x in t[t == 'int64'].index:
    data[x] = data[x].astype('int32')
for x in t[t == 'float64'].index:
    data[x] = data[x].astype('float32')
    
feature_cols_v5 = [x for x in data.columns if x not in ['uid', 'qid', 'title_t2', 'hist_title_t2', 'desc_t2', 'hist_desc_t2']]
logging.info('feature_cols: %s', feature_cols_v5)

[2019-12-14 18:48:08,395] INFO: feature_cols: ['a', 'b', 'c', 'd', 'e', 'aa', 'bb', 'cc', 'dd', 'ee']


In [23]:
# 赶快存变量
X_train_all_v5 = data.iloc[:train_size][feature_cols_v5]
test_v5 = data.iloc[train_size:][feature_cols_v5]
test_v5 = test_v5.reset_index()

gdump(X_train_all_v5, 'X_train_all_v5')
gdump(test_v5, 'test_v5')
gdump(feature_cols_v5, 'feature_cols_v5')

In [24]:
X_train_all_v5

,a,b,c,d,e,aa,bb,cc,dd,ee
0,0,0.0,0.000000,0,0.000000,0,0.0,0.0,0,0.000000
1,9,1.0,0.750000,23,0.209091,12,1.0,1.0,45,0.173077
2,0,0.0,0.000000,0,0.000000,0,0.0,0.0,0,0.000000
3,7,1.0,0.583333,16,0.064257,0,0.0,0.0,0,0.000000
4,10,1.0,0.833333,18,0.008751,12,1.0,1.0,56,0.012267
...,...,...,...,...,...,...,...,...,...,...
2593664,0,0.0,0.000000,0,0.000000,0,0.0,0.0,0,0.000000
2593665,11,1.0,0.916667,30,0.120968,0,0.0,0.0,0,0.000000
2593666,10,1.0,0.833333,20,0.080645,12,1.0,1.0,97,1.000000
2593667,11,1.0,0.916667,36,0.014057,0,0.0,0.0,0,0.000000


In [30]:
del data

# uhist数据训练

## 导入包、超参数定义

In [3]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten

import os
import tensorflow as tf
from keras.backend import set_session
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
config = tf.compat.v1.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.8  # 占用GPU百分比
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config)) # 此处不同

BATCH_SIZE = 8192
EPOCHS = 5
LR = 0.01

callbacks=[
    tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', 
                                         factor=0.1, 
                                         patience=3, 
                                         verbose=1, 
                                         mode='auto', 
                                         min_delta=0.0001, 
                                         cooldown=0, 
                                         min_lr=0),  # 当指标变化小时，减少学习率
]

Using TensorFlow backend.


## 数据处理

准备数据。

In [3]:
X_train_all_v5 = gload('X_train_all_v5')
y_train_all = gload('y_train_all')

In [5]:
y_train_all

0          0.0
1          0.0
2          0.0
3          0.0
4          1.0
          ... 
2593664    0.0
2593665    0.0
2593666    1.0
2593667    1.0
2593668    0.0
Name: label, Length: 2593669, dtype: float32

归一化数据。

In [6]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
x = X_train_all_v5.values
x = mms.fit_transform(x)
X_train_all_v5 = pd.DataFrame(x)
del x
X_train_all_v5[:5]

,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
1,0.750000,1.0,0.750000,0.097872,0.017139,1.0,1.0,1.0,0.007606,0.000648
2,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
3,0.583333,1.0,0.583333,0.068085,0.005267,0.0,0.0,0.0,0.000000,0.000000
4,0.833333,1.0,0.833333,0.076596,0.000717,1.0,1.0,1.0,0.009466,0.000046


过采样均衡数据。

In [7]:
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

logging.info('y_train distribution is %s' % Counter(y_train_all))
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train_all_v5, y_train_all)
logging.info('y_resampled distribution is %s' % Counter(y_resampled))

# del X_train_all_v5, y_train_all

[2019-12-15 15:27:14,407] INFO: y_train distribution is Counter({0.0: 2178656, 1.0: 415013})
[2019-12-15 15:27:18,272] INFO: y_resampled distribution is Counter({0.0: 2178656, 1.0: 2178656})


## NN训练

In [15]:
model_2 = Sequential()

model_2.add(Dense(256, activation='tanh', input_shape=(X_resampled.shape[1],)))
model_2.add(Dropout(0.3))
model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.3))
model_2.add(Dense(512, activation='tanh'))
model_2.add(Dropout(0.3))
model_2.add(Dense(256, activation='linear'))
model_2.add(Dense(1, activation='sigmoid'))

model_2.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# model.summary()

In [9]:
X_resampled = X_resampled.values
y_resampled = y_resampled.values
np.random.seed(100)
np.random.shuffle(X_resampled)
np.random.seed(100)
np.random.shuffle(y_resampled)

In [16]:
H = model_2.fit(X_resampled, y_resampled, 
              epochs=100, batch_size=BATCH_SIZE, #validation_split=0.1,
              shuffle='batch', callbacks=callbacks, verbose=1)

Epoch 1/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6675 - accuracy: 0.5912
Epoch 2/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.6436 - accuracy: 0.6196
Epoch 3/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.6351 - accuracy: 0.6273
Epoch 4/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.6311 - accuracy: 0.6313
Epoch 5/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6286 - accuracy: 0.6337
Epoch 6/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.6269 - accuracy: 0.6354
Epoch 7/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.6256 - accuracy: 0.6363
Epoch 8/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.6247 - accuracy: 0.6373
Epoch 9/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6237 - accuracy: 0.6380
Epoch 10/100
435731

In [17]:
gdump(model_2, 'model_colv5')

In [63]:
model_2 = gload('model_colv5')

In [18]:
del X_resampled, y_resampled

## 计算colv5

In [36]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
x = X_train_all_v5.values
x = mms.fit_transform(x)
X_train_all_v5 = pd.DataFrame(x)
del x
X_train_all_v5[:5]

,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
1,0.750000,1.0,0.750000,0.097872,0.017139,1.0,1.0,1.0,0.007606,0.000648
2,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
3,0.583333,1.0,0.583333,0.068085,0.005267,0.0,0.0,0.0,0.000000,0.000000
4,0.833333,1.0,0.833333,0.076596,0.000717,1.0,1.0,1.0,0.009466,0.000046


In [19]:
# del X_resampled, y_resampled
colv5 = model_2.predict(X_train_all_v5.values, batch_size=BATCH_SIZE)
colv5 = pd.DataFrame(colv5)
colv5.columns = ['colv5']
colv5[:5], colv5.shape

(      colv5
 0  0.329246
 1  0.483628
 2  0.329246
 3  0.579087
 4  0.680988, (2593669, 1))

In [20]:
y_train_all = gload('y_train_all')
model_2.evaluate(X_train_all_v5.values, y_train_all.values, batch_size=BATCH_SIZE)

2593669/2593669 [==============================] - 2s 1us/step


[0.5925416356934442, 0.6364497542381287]

In [21]:
gdump(colv5, 'colv5')

In [22]:
del X_train_all_v5

# 训练

## 取出数据

In [23]:
X_train_all = read_param('X_train_all')
y_train_all = gload('y_train_all')
colv5 = gload('colv5')

# 拼接数据
X_train_all = pd.concat([X_train_all, colv5['colv5']], axis=1)
del colv5
X_train_all

,freq,gender,hour,q_ans_count,q_diff_qa_days_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_min,q_diff_qa_days_std,q_diff_qa_days_sum,...,qid_enc_count,gender_count,freq_count,uf_c1_count,uf_c2_count,uf_c3_count,uf_c4_count,uf_c5_count,wk,colv5
0,5,3,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000369,1.000000,0.961323,0.022204,0.371510,0.217985,0.062753,1.0,1,0.329246
1,5,3,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000036,1.000000,0.961323,0.024503,0.111695,0.443770,0.002973,1.0,5,0.483628
2,2,3,4,32.0,32.0,13.0,9.531250,0.0,4.634961,305.0,...,0.001341,1.000000,0.291847,0.006308,0.034012,0.400961,0.044143,1.0,3,0.329246
3,1,2,20,3.0,3.0,234.0,222.333328,215.0,10.214369,667.0,...,0.000018,0.000000,1.000000,0.024503,0.111695,0.443770,0.002973,1.0,4,0.579087
4,5,1,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000027,0.009572,0.961323,0.005711,0.007464,0.438832,0.126329,1.0,5,0.680988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2593664,1,1,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000045,0.009572,1.000000,1.000000,1.000000,0.121446,1.000000,1.0,0,0.329246
2593665,1,3,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000180,1.000000,1.000000,1.000000,1.000000,0.227937,1.000000,1.0,5,0.507993
2593666,1,3,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000036,1.000000,1.000000,1.000000,1.000000,0.227937,1.000000,1.0,3,0.727206
2593667,1,2,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000054,0.000000,1.000000,0.007801,0.068926,0.443770,0.002962,1.0,5,0.648089


In [17]:
# # 取出过一次后再取就不用处理啦
# X_train_all = gload('X_trian_v5_all_reduced')
# y_train_all = gload('y_train_all')

Compress data.（超强压缩算法）

In [22]:
# # @from: https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65/code
# # @liscense: Apache 2.0
# # @author: weijian
# def reduce_mem_usage(props):
#     # 计算当前内存
#     start_mem_usg = props.memory_usage().sum() / 1024 ** 2
#     print("Memory usage of the dataframe is :", start_mem_usg, "MB")
    
#     # 哪些列包含空值，空值用-999填充。why：因为np.nan当做float处理
#     NAlist = []
#     for col in props.columns:
#         # 这里只过滤了objectd格式，如果你的代码中还包含其他类型，请一并过滤
#         if (props[col].dtypes != object):
            
#             print("**************************")
#             print("columns: ", col)
#             print("dtype before", props[col].dtype)
            
#             # 判断是否是int类型
#             isInt = False
#             mmax = props[col].max()
#             mmin = props[col].min()
            
#             # Integer does not support NA, therefore Na needs to be filled
#             if not np.isfinite(props[col]).all():
#                 NAlist.append(col)
#                 props[col].fillna(-999, inplace=True) # 用-999填充
                
#             # test if column can be converted to an integer
#             asint = props[col].fillna(0).astype(np.int64)
#             result = np.fabs(props[col] - asint)
#             result = result.sum()
#             if result < 0.01: # 绝对误差和小于0.01认为可以转换的，要根据task修改
#                 isInt = True
            
#             # make interger / unsigned Integer datatypes
#             if isInt:
#                 if mmin >= 0: # 最小值大于0，转换成无符号整型
#                     if mmax <= 255:
#                         props[col] = props[col].astype(np.uint8)
#                     elif mmax <= 65535:
#                         props[col] = props[col].astype(np.uint16)
#                     elif mmax <= 4294967295:
#                         props[col] = props[col].astype(np.uint32)
#                     else:
#                         props[col] = props[col].astype(np.uint64)
#                 else: # 转换成有符号整型
#                     if mmin > np.iinfo(np.int8).min and mmax < np.iinfo(np.int8).max:
#                         props[col] = props[col].astype(np.int8)
#                     elif mmin > np.iinfo(np.int16).min and mmax < np.iinfo(np.int16).max:
#                         props[col] = props[col].astype(np.int16)
#                     elif mmin > np.iinfo(np.int32).min and mmax < np.iinfo(np.int32).max:
#                         props[col] = props[col].astype(np.int32)
#                     elif mmin > np.iinfo(np.int64).min and mmax < np.iinfo(np.int64).max:
#                         props[col] = props[col].astype(np.int64)  
#             else: # 注意：这里对于float都转换成float16，需要根据你的情况自己更改
#                 props[col] = props[col].astype(np.float32)
            
#             print("dtype after", props[col].dtype)
#             print("********************************")
#     print("___MEMORY USAGE AFTER COMPLETION:___")
#     mem_usg = props.memory_usage().sum() / 1024**2 
#     print("Memory usage is: ",mem_usg," MB")
#     print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
#     return props, NAlist

In [21]:
# reduce_mem_usage(X_train_all)

Memory usage of the dataframe is : 2306.163959503174 MB
**************************
columns:  freq
dtype before int32
dtype after uint8
********************************
**************************
columns:  gender
dtype before int32
dtype after uint8
********************************
**************************
columns:  hour
dtype before int32
dtype after uint8
********************************
**************************
columns:  q_ans_count
dtype before float32
dtype after uint16
********************************
**************************
columns:  q_diff_qa_days_count
dtype before float32
dtype after uint16
********************************
**************************
columns:  q_diff_qa_days_max
dtype before float32
dtype after int16
********************************
**************************
columns:  q_diff_qa_days_mean
dtype before float32
dtype after float32
********************************
**************************
columns:  q_diff_qa_days_min
dtype before float32
dtype after int16

dtype after uint32
********************************
**************************
columns:  q_reci_mark_mean
dtype before float32
dtype after float32
********************************
**************************
columns:  q_reci_mark_min
dtype before float32
dtype after uint16
********************************
**************************
columns:  q_reci_mark_std
dtype before float32
dtype after float32
********************************
**************************
columns:  q_reci_mark_sum
dtype before float32
dtype after uint32
********************************
**************************
columns:  q_reci_no_help_count
dtype before float32
dtype after uint16
********************************
**************************
columns:  q_reci_no_help_max
dtype before float32
dtype after uint8
********************************
**************************
columns:  q_reci_no_help_mean
dtype before float32
dtype after float32
********************************
**************************
columns:  q_reci_no_help

dtype after uint8
********************************
**************************
columns:  u_is_dest_sum
dtype before float32
dtype after uint8
********************************
**************************
columns:  u_is_good_count
dtype before float32
dtype after uint16
********************************
**************************
columns:  u_is_good_max
dtype before float32
dtype after uint8
********************************
**************************
columns:  u_is_good_mean
dtype before float32
dtype after float32
********************************
**************************
columns:  u_is_good_min
dtype before float32
dtype after uint8
********************************
**************************
columns:  u_is_good_std
dtype before float32
dtype after float32
********************************
**************************
columns:  u_is_good_sum
dtype before float32
dtype after uint8
********************************
**************************
columns:  u_is_rec_count
dtype before float32
dtype a

dtype after uint16
********************************
**************************
columns:  u_word_count_std
dtype before float32
dtype after float32
********************************
**************************
columns:  u_word_count_sum
dtype before float32
dtype after uint32
********************************
**************************
columns:  uf_b1
dtype before int32
dtype after uint8
********************************
**************************
columns:  uf_b2
dtype before int32
dtype after uint8
********************************
**************************
columns:  uf_b3
dtype before int32
dtype after uint8
********************************
**************************
columns:  uf_b4
dtype before int32
dtype after uint8
********************************
**************************
columns:  uf_b5
dtype before int32
dtype after uint8
********************************
**************************
columns:  uf_c1
dtype before int32
dtype after uint16
********************************
**************

(         freq  gender  hour  q_ans_count  q_diff_qa_days_count  \
 0           5       3    22        64537                 64537   
 1           5       3    15        64537                 64537   
 2           2       3     4           32                    32   
 3           1       2    20            3                     3   
 4           5       1     8        64537                 64537   
 ...       ...     ...   ...          ...                   ...   
 2593664     1       1    20        64537                 64537   
 2593665     1       3    15        64537                 64537   
 2593666     1       3    18        64537                 64537   
 2593667     1       2    12        64537                 64537   
 2593668     1       2     7        64537                 64537   
 
          q_diff_qa_days_max  q_diff_qa_days_mean  q_diff_qa_days_min  \
 0                      -999          -999.000000                -999   
 1                      -999          -999.00000

In [50]:
# gdump(X_train_all, 'X_trian_v5_all_reduced')

## LGBM分类器训练

### cv交叉验证

In [26]:
# 221 input
X_train_all = read_param('X_train_all')
y_train_all = gload('y_train_all')
feature_cols = gload('feature_cols')
feature_cols.append('colv5')
print('X_train_all.shape:', X_train_all.shape)

X_tmp, y_tmp = X_train_all[int(0.2*len(X_train_all)):], y_train_all[int(0.2*len(X_train_all)):]
train_data = lgb.Dataset(X_tmp[feature_cols], y_tmp)
del X_train_all, y_train_all

model_lgb = LGBMClassifier(boosting_type='dart',
                           objective='binary', 
                           learning_rate=0.3,
                           n_estimators=200, 
                           max_depth=20,
                           num_leaves=190,
                           min_child_samples=210,
                           min_child_weight=0.055,
                           reg_alpha=0.005,
                           reg_lambda=21,
                           n_jobs=16,
                           seed=666, 
                           silent=True,
                           device='gpu',
                          )

cv_res = lgb.cv(model_lgb.get_params(), train_data, num_boost_round=5,
                nfold=5, metrics=['logloss', 'auc'], verbose_eval=5,
                early_stopping_rounds=10, stratified=False, shuffle=True,)

print('best n_estimators:', len(cv_res['auc-mean']))
print('best cv score:', cv_res['auc-mean'][-1])
del X_tmp, y_tmp, train_data

[5]	cv_agg's auc: 0.865387 + 0.00112399
[10]	cv_agg's auc: 0.869967 + 0.00107055
[15]	cv_agg's auc: 0.874106 + 0.000953568
[20]	cv_agg's auc: 0.876051 + 0.00102926
[25]	cv_agg's auc: 0.87731 + 0.00103074
[30]	cv_agg's auc: 0.878381 + 0.000978689
[35]	cv_agg's auc: 0.878787 + 0.000996533
[40]	cv_agg's auc: 0.879695 + 0.000988646
[45]	cv_agg's auc: 0.880335 + 0.000954589
[50]	cv_agg's auc: 0.880247 + 0.000953845
[55]	cv_agg's auc: 0.880491 + 0.000934943
[60]	cv_agg's auc: 0.881237 + 0.000896446
[65]	cv_agg's auc: 0.881617 + 0.000840497
[70]	cv_agg's auc: 0.882076 + 0.000803534
[75]	cv_agg's auc: 0.882436 + 0.000809805
[80]	cv_agg's auc: 0.882754 + 0.000792338
[85]	cv_agg's auc: 0.883035 + 0.000822719
[90]	cv_agg's auc: 0.883287 + 0.000864668
[95]	cv_agg's auc: 0.883404 + 0.000848585
[100]	cv_agg's auc: 0.883607 + 0.000828861
[105]	cv_agg's auc: 0.883681 + 0.000831901
[110]	cv_agg's auc: 0.883782 + 0.000836735
[115]	cv_agg's auc: 0.883881 + 0.000820045
[120]	cv_agg's auc: 0.88395 + 0.0008

In [48]:
# 230 input
X_train_all = read_param('X_train_all')
X_train_all_v5 = gload('X_train_all_v5')
y_train_all = gload('y_train_all')
feature_cols = gload('feature_cols')
feature_cols_v5 = gload('feature_cols_v5')

X_train_all = pd.concat([X_train_all, X_train_all_v5[feature_cols_v5]], axis=1)
feature_cols.extend(feature_cols_v5)
print('X_train_all.shape:', X_train_all.shape)
del X_train_all_v5, feature_cols_v5

X_tmp, y_tmp = X_train_all[int(0.2*len(X_train_all)):], y_train_all[int(0.2*len(X_train_all)):]
train_data = lgb.Dataset(X_tmp, y_tmp)
del X_train_all, y_train_all

model_lgb = LGBMClassifier(boosting_type='dart',
                           objective='binary', 
                           learning_rate=0.3,
                           n_estimators=200, 
                           max_depth=20,
                           num_leaves=190,
                           min_child_samples=210,
                           min_child_weight=0.055,
                           reg_alpha=0.005,
                           reg_lambda=21,
                           n_jobs=16,
                           seed=666, 
                           silent=True,
                           device='gpu',
                          )

cv_res = lgb.cv(model_lgb.get_params(), train_data, num_boost_round=5,
                nfold=5, metrics=['logloss', 'auc'], verbose_eval=5,
                early_stopping_rounds=10, stratified=False, shuffle=True,)

print('best n_estimators:', len(cv_res['auc-mean']))
print('best cv score:', cv_res['auc-mean'][-1])
del X_tmp, y_tmp, train_data

[5]	cv_agg's auc: 0.864006 + 0.00107252
[10]	cv_agg's auc: 0.870316 + 0.0011868
[15]	cv_agg's auc: 0.876153 + 0.00095644
[20]	cv_agg's auc: 0.878644 + 0.000949346
[25]	cv_agg's auc: 0.880628 + 0.00088579
[30]	cv_agg's auc: 0.881883 + 0.000844579
[35]	cv_agg's auc: 0.882549 + 0.000795606
[40]	cv_agg's auc: 0.883765 + 0.00078133
[45]	cv_agg's auc: 0.884832 + 0.000801308
[50]	cv_agg's auc: 0.884714 + 0.000812119
[55]	cv_agg's auc: 0.885051 + 0.000931006
[60]	cv_agg's auc: 0.88604 + 0.000836551
[65]	cv_agg's auc: 0.886661 + 0.000752837
[70]	cv_agg's auc: 0.887247 + 0.000701844
[75]	cv_agg's auc: 0.887777 + 0.000717053
[80]	cv_agg's auc: 0.888298 + 0.000720959
[85]	cv_agg's auc: 0.888662 + 0.000723602
[90]	cv_agg's auc: 0.88903 + 0.000734121
[95]	cv_agg's auc: 0.88912 + 0.000723676
[100]	cv_agg's auc: 0.889406 + 0.000693491
[105]	cv_agg's auc: 0.889534 + 0.00073621
[110]	cv_agg's auc: 0.889704 + 0.00075198
[115]	cv_agg's auc: 0.889847 + 0.000714147
[120]	cv_agg's auc: 0.889894 + 0.000712545

In [49]:
# 231 input
X_train_all = read_param('X_train_all')
X_train_all_v5 = gload('X_train_all_v5')
y_train_all = gload('y_train_all')
feature_cols = gload('feature_cols')
feature_cols_v5 = gload('feature_cols_v5')
colv5 = gload('colv5')

X_train_all = pd.concat([X_train_all, X_train_all_v5[feature_cols_v5]], axis=1)
X_train_all = pd.concat([X_train_all, colv5['colv5']], axis=1)
feature_cols.extend(feature_cols_v5)
print('X_train_all.shape:', X_train_all.shape)
del X_train_all_v5, feature_cols_v5, colv5

X_tmp, y_tmp = X_train_all[int(0.2*len(X_train_all)):], y_train_all[int(0.2*len(X_train_all)):]
train_data = lgb.Dataset(X_tmp[feature_cols], y_tmp)
del X_train_all, y_train_all

model_lgb = LGBMClassifier(boosting_type='dart',
                           objective='binary', 
                           learning_rate=0.3,
                           n_estimators=200, 
                           max_depth=20,
                           num_leaves=190,
                           min_child_samples=210,
                           min_child_weight=0.055,
                           reg_alpha=0.005,
                           reg_lambda=21,
                           n_jobs=16,
                           seed=666, 
                           silent=True,
                           device='gpu',
                          )

cv_res = lgb.cv(model_lgb.get_params(), train_data, num_boost_round=5,
                nfold=5, metrics=['logloss', 'auc'], verbose_eval=5,
                early_stopping_rounds=10, stratified=False, shuffle=True,)

print('best n_estimators:', len(cv_res['auc-mean']))
print('best cv score:', cv_res['auc-mean'][-1])
del X_tmp, y_tmp, train_data

X_train_all.shape: (2593669, 231)
[5]	cv_agg's auc: 0.866971 + 0.000909077
[10]	cv_agg's auc: 0.87246 + 0.000856691
[15]	cv_agg's auc: 0.877834 + 0.000777032
[20]	cv_agg's auc: 0.880093 + 0.000790816
[25]	cv_agg's auc: 0.881775 + 0.000788219
[30]	cv_agg's auc: 0.883058 + 0.000791931
[35]	cv_agg's auc: 0.88354 + 0.000874643
[40]	cv_agg's auc: 0.884897 + 0.000980263
[45]	cv_agg's auc: 0.885874 + 0.000975463
[50]	cv_agg's auc: 0.885703 + 0.000943437
[55]	cv_agg's auc: 0.885944 + 0.000953915
[60]	cv_agg's auc: 0.886889 + 0.000866099
[65]	cv_agg's auc: 0.887457 + 0.000829388
[70]	cv_agg's auc: 0.888021 + 0.000874698
[75]	cv_agg's auc: 0.888495 + 0.000861532
[80]	cv_agg's auc: 0.888985 + 0.000867859
[85]	cv_agg's auc: 0.889338 + 0.000837581
[90]	cv_agg's auc: 0.889612 + 0.000772813
[95]	cv_agg's auc: 0.889727 + 0.000797705
[100]	cv_agg's auc: 0.889909 + 0.000760363
[105]	cv_agg's auc: 0.890038 + 0.000816991
[110]	cv_agg's auc: 0.890129 + 0.000803543
[115]	cv_agg's auc: 0.890246 + 0.000773463

### train

In [50]:
# 231 input
X_train_all = read_param('X_train_all')
X_train_all_v5 = gload('X_train_all_v5')
y_train_all = gload('y_train_all')
feature_cols = gload('feature_cols')
feature_cols_v5 = gload('feature_cols_v5')
colv5 = gload('colv5')

X_train_all = pd.concat([X_train_all, X_train_all_v5[feature_cols_v5]], axis=1)
X_train_all = pd.concat([X_train_all, colv5['colv5']], axis=1)
feature_cols.extend(feature_cols_v5)
print('X_train_all.shape:', X_train_all.shape)
del X_train_all_v5, feature_cols_v5, colv5

X_train_all.shape: (2593669, 231)


In [51]:
X_val = X_train_all[:50000]
y_val = y_train_all[:50000]

In [52]:
model_lgb = LGBMClassifier(boosting_type='dart',
                           objective='binary', 
                           learning_rate=0.005,
                           n_estimators=10000, 
                           max_depth=20,
                           num_leaves=190,
                           min_child_samples=210,
                           min_child_weight=0.055,
                           reg_alpha=0.005,
                           reg_lambda=21,
                           n_jobs=16,
#                            seed=666, 
                           silent=True,
                           device='gpu',)
model_lgb.fit(X_train_all, y_train_all,
              eval_metric=['logloss', 'auc'],
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=50)

[1]	valid_0's auc: 0.847349	valid_0's binary_logloss: 0.437532
[2]	valid_0's auc: 0.847627	valid_0's binary_logloss: 0.43615
[3]	valid_0's auc: 0.848124	valid_0's binary_logloss: 0.434795
[4]	valid_0's auc: 0.848158	valid_0's binary_logloss: 0.433469
[5]	valid_0's auc: 0.848158	valid_0's binary_logloss: 0.432169
[6]	valid_0's auc: 0.848238	valid_0's binary_logloss: 0.430893
[7]	valid_0's auc: 0.848357	valid_0's binary_logloss: 0.429641
[8]	valid_0's auc: 0.84841	valid_0's binary_logloss: 0.430088
[9]	valid_0's auc: 0.8494	valid_0's binary_logloss: 0.428847
[10]	valid_0's auc: 0.850565	valid_0's binary_logloss: 0.427619
[11]	valid_0's auc: 0.851288	valid_0's binary_logloss: 0.426408
[12]	valid_0's auc: 0.851313	valid_0's binary_logloss: 0.426826
[13]	valid_0's auc: 0.851662	valid_0's binary_logloss: 0.425636
[14]	valid_0's auc: 0.852017	valid_0's binary_logloss: 0.424458
[15]	valid_0's auc: 0.85228	valid_0's binary_logloss: 0.423301
[16]	valid_0's auc: 0.852805	valid_0's binary_logloss:

[130]	valid_0's auc: 0.857453	valid_0's binary_logloss: 0.445021
[131]	valid_0's auc: 0.857449	valid_0's binary_logloss: 0.445418
[132]	valid_0's auc: 0.857473	valid_0's binary_logloss: 0.444443
[133]	valid_0's auc: 0.857509	valid_0's binary_logloss: 0.443475
[134]	valid_0's auc: 0.857517	valid_0's binary_logloss: 0.451028
[135]	valid_0's auc: 0.857563	valid_0's binary_logloss: 0.450011
[136]	valid_0's auc: 0.857573	valid_0's binary_logloss: 0.460367
[137]	valid_0's auc: 0.857573	valid_0's binary_logloss: 0.460802
[138]	valid_0's auc: 0.857639	valid_0's binary_logloss: 0.459714
[139]	valid_0's auc: 0.857689	valid_0's binary_logloss: 0.458639
[140]	valid_0's auc: 0.857695	valid_0's binary_logloss: 0.459186
[141]	valid_0's auc: 0.857689	valid_0's binary_logloss: 0.459611
[142]	valid_0's auc: 0.857678	valid_0's binary_logloss: 0.460063
[143]	valid_0's auc: 0.857703	valid_0's binary_logloss: 0.45898
[144]	valid_0's auc: 0.857707	valid_0's binary_logloss: 0.459358
[145]	valid_0's auc: 0.857

[257]	valid_0's auc: 0.859957	valid_0's binary_logloss: 0.448435
[258]	valid_0's auc: 0.859994	valid_0's binary_logloss: 0.447435
[259]	valid_0's auc: 0.859991	valid_0's binary_logloss: 0.447986
[260]	valid_0's auc: 0.859989	valid_0's binary_logloss: 0.448591
[261]	valid_0's auc: 0.859986	valid_0's binary_logloss: 0.449128
[262]	valid_0's auc: 0.860003	valid_0's binary_logloss: 0.448122
[263]	valid_0's auc: 0.860039	valid_0's binary_logloss: 0.447131
[264]	valid_0's auc: 0.860063	valid_0's binary_logloss: 0.446143
[265]	valid_0's auc: 0.860084	valid_0's binary_logloss: 0.445161
[266]	valid_0's auc: 0.860135	valid_0's binary_logloss: 0.444187
[267]	valid_0's auc: 0.860134	valid_0's binary_logloss: 0.444728
[268]	valid_0's auc: 0.860171	valid_0's binary_logloss: 0.443752
[269]	valid_0's auc: 0.860167	valid_0's binary_logloss: 0.444303
[270]	valid_0's auc: 0.860169	valid_0's binary_logloss: 0.443337
[271]	valid_0's auc: 0.860157	valid_0's binary_logloss: 0.443898
[272]	valid_0's auc: 0.86

[384]	valid_0's auc: 0.861882	valid_0's binary_logloss: 0.4262
[385]	valid_0's auc: 0.861875	valid_0's binary_logloss: 0.426742
[386]	valid_0's auc: 0.861921	valid_0's binary_logloss: 0.425888
[387]	valid_0's auc: 0.861952	valid_0's binary_logloss: 0.425044
[388]	valid_0's auc: 0.861968	valid_0's binary_logloss: 0.424206
[389]	valid_0's auc: 0.862013	valid_0's binary_logloss: 0.423372
[390]	valid_0's auc: 0.862044	valid_0's binary_logloss: 0.422545
[391]	valid_0's auc: 0.862037	valid_0's binary_logloss: 0.42306
[392]	valid_0's auc: 0.862064	valid_0's binary_logloss: 0.422236
[393]	valid_0's auc: 0.862094	valid_0's binary_logloss: 0.421419
[394]	valid_0's auc: 0.862132	valid_0's binary_logloss: 0.420603
[395]	valid_0's auc: 0.86217	valid_0's binary_logloss: 0.419793
[396]	valid_0's auc: 0.862166	valid_0's binary_logloss: 0.420271
[397]	valid_0's auc: 0.862182	valid_0's binary_logloss: 0.419467
[398]	valid_0's auc: 0.862223	valid_0's binary_logloss: 0.418668
[399]	valid_0's auc: 0.862291

[511]	valid_0's auc: 0.864062	valid_0's binary_logloss: 0.393431
[512]	valid_0's auc: 0.864086	valid_0's binary_logloss: 0.392805
[513]	valid_0's auc: 0.864106	valid_0's binary_logloss: 0.392178
[514]	valid_0's auc: 0.864151	valid_0's binary_logloss: 0.391554
[515]	valid_0's auc: 0.864167	valid_0's binary_logloss: 0.390939
[516]	valid_0's auc: 0.864165	valid_0's binary_logloss: 0.39132
[517]	valid_0's auc: 0.864164	valid_0's binary_logloss: 0.391708
[518]	valid_0's auc: 0.864165	valid_0's binary_logloss: 0.39203
[519]	valid_0's auc: 0.864164	valid_0's binary_logloss: 0.392429
[520]	valid_0's auc: 0.864202	valid_0's binary_logloss: 0.391806
[521]	valid_0's auc: 0.86424	valid_0's binary_logloss: 0.391193
[522]	valid_0's auc: 0.864233	valid_0's binary_logloss: 0.391596
[523]	valid_0's auc: 0.86423	valid_0's binary_logloss: 0.391998
[524]	valid_0's auc: 0.864228	valid_0's binary_logloss: 0.392407
[525]	valid_0's auc: 0.864227	valid_0's binary_logloss: 0.392784
[526]	valid_0's auc: 0.864237

...

[9519]	valid_0's auc: 0.901167	valid_0's binary_logloss: 0.265034
[9520]	valid_0's auc: 0.901169	valid_0's binary_logloss: 0.265028
[9521]	valid_0's auc: 0.90117	valid_0's binary_logloss: 0.26503
[9522]	valid_0's auc: 0.901171	valid_0's binary_logloss: 0.265033
[9523]	valid_0's auc: 0.901173	valid_0's binary_logloss: 0.265025
[9524]	valid_0's auc: 0.901174	valid_0's binary_logloss: 0.265028
[9525]	valid_0's auc: 0.901175	valid_0's binary_logloss: 0.265031
[9526]	valid_0's auc: 0.901177	valid_0's binary_logloss: 0.265025
[9527]	valid_0's auc: 0.901178	valid_0's binary_logloss: 0.265029
[9528]	valid_0's auc: 0.901179	valid_0's binary_logloss: 0.265032
[9529]	valid_0's auc: 0.901181	valid_0's binary_logloss: 0.265026
[9530]	valid_0's auc: 0.901183	valid_0's binary_logloss: 0.26502
[9531]	valid_0's auc: 0.901186	valid_0's binary_logloss: 0.265013
[9532]	valid_0's auc: 0.901188	valid_0's binary_logloss: 0.265008
[9533]	valid_0's auc: 0.901189	valid_0's binary_logloss: 0.26501
[9534]	valid_0

[9644]	valid_0's auc: 0.901354	valid_0's binary_logloss: 0.264783
[9645]	valid_0's auc: 0.901355	valid_0's binary_logloss: 0.264786
[9646]	valid_0's auc: 0.901356	valid_0's binary_logloss: 0.264781
[9647]	valid_0's auc: 0.901359	valid_0's binary_logloss: 0.264774
[9648]	valid_0's auc: 0.90136	valid_0's binary_logloss: 0.264775
[9649]	valid_0's auc: 0.901363	valid_0's binary_logloss: 0.264769
[9650]	valid_0's auc: 0.901364	valid_0's binary_logloss: 0.264771
[9651]	valid_0's auc: 0.901365	valid_0's binary_logloss: 0.264775
[9652]	valid_0's auc: 0.901365	valid_0's binary_logloss: 0.264777
[9653]	valid_0's auc: 0.901368	valid_0's binary_logloss: 0.264769
[9654]	valid_0's auc: 0.901369	valid_0's binary_logloss: 0.264763
[9655]	valid_0's auc: 0.90137	valid_0's binary_logloss: 0.264766
[9656]	valid_0's auc: 0.901373	valid_0's binary_logloss: 0.264759
[9657]	valid_0's auc: 0.901374	valid_0's binary_logloss: 0.264762
[9658]	valid_0's auc: 0.901376	valid_0's binary_logloss: 0.264756
[9659]	valid

[9769]	valid_0's auc: 0.901553	valid_0's binary_logloss: 0.264502
[9770]	valid_0's auc: 0.901557	valid_0's binary_logloss: 0.264496
[9771]	valid_0's auc: 0.901559	valid_0's binary_logloss: 0.264502
[9772]	valid_0's auc: 0.901559	valid_0's binary_logloss: 0.264504
[9773]	valid_0's auc: 0.901561	valid_0's binary_logloss: 0.264507
[9774]	valid_0's auc: 0.901561	valid_0's binary_logloss: 0.264509
[9775]	valid_0's auc: 0.901563	valid_0's binary_logloss: 0.264512
[9776]	valid_0's auc: 0.901563	valid_0's binary_logloss: 0.264514
[9777]	valid_0's auc: 0.901565	valid_0's binary_logloss: 0.264507
[9778]	valid_0's auc: 0.901568	valid_0's binary_logloss: 0.264501
[9779]	valid_0's auc: 0.90157	valid_0's binary_logloss: 0.264495
[9780]	valid_0's auc: 0.901572	valid_0's binary_logloss: 0.264489
[9781]	valid_0's auc: 0.901573	valid_0's binary_logloss: 0.264483
[9782]	valid_0's auc: 0.901575	valid_0's binary_logloss: 0.264475
[9783]	valid_0's auc: 0.901577	valid_0's binary_logloss: 0.264479
[9784]	vali

[9894]	valid_0's auc: 0.901742	valid_0's binary_logloss: 0.264268
[9895]	valid_0's auc: 0.901745	valid_0's binary_logloss: 0.264261
[9896]	valid_0's auc: 0.901747	valid_0's binary_logloss: 0.264263
[9897]	valid_0's auc: 0.901748	valid_0's binary_logloss: 0.264257
[9898]	valid_0's auc: 0.90175	valid_0's binary_logloss: 0.264251
[9899]	valid_0's auc: 0.901751	valid_0's binary_logloss: 0.264253
[9900]	valid_0's auc: 0.901753	valid_0's binary_logloss: 0.264248
[9901]	valid_0's auc: 0.901756	valid_0's binary_logloss: 0.264242
[9902]	valid_0's auc: 0.901757	valid_0's binary_logloss: 0.264237
[9903]	valid_0's auc: 0.901759	valid_0's binary_logloss: 0.26423
[9904]	valid_0's auc: 0.901764	valid_0's binary_logloss: 0.264223
[9905]	valid_0's auc: 0.901765	valid_0's binary_logloss: 0.264225
[9906]	valid_0's auc: 0.901766	valid_0's binary_logloss: 0.264227
[9907]	valid_0's auc: 0.901767	valid_0's binary_logloss: 0.264223
[9908]	valid_0's auc: 0.901768	valid_0's binary_logloss: 0.264217
[9909]	valid

LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
               device='gpu', importance_type='split', learning_rate=0.005,
               max_depth=20, min_child_samples=210, min_child_weight=0.055,
               min_split_gain=0.0, n_estimators=10000, n_jobs=16,
               num_leaves=190, objective='binary', random_state=None,
               reg_alpha=0.005, reg_lambda=21, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [53]:
gdump(model_lgb, 'model_lgb_231')

In [55]:
del X_train_all

## 集成学习Ensemble

In [31]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten

import os
import tensorflow as tf
from keras.backend import set_session
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
config = tf.compat.v1.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.8  # 占用GPU百分比
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config)) # 此处不同

BATCH_SIZE = 8192
EPOCHS = 5
LR = 0.01

callbacks=[
    tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', 
                                         factor=0.1, 
                                         patience=3, 
                                         verbose=1, 
                                         mode='auto', 
                                         min_delta=0.0001, 
                                         cooldown=0, 
                                         min_lr=0),  # 当指标变化小时，减少学习率
]

In [34]:
colv5 = gload('colv5')
feature_cols = gload('feature_cols')
model_v4 = gload('model_v4')
X_train_all = read_param('X_train_all')
y_train_all = gload('y_train_all')

In [35]:
X_train = pd.DataFrame()
X_train['v4'] = model_v4.predict_proba(X_train_all[feature_cols[:220]])[:, 1]
X_train['v5'] = colv5
y_train = y_train_all
X_train

,v4,v5
0,0.006213,0.329246
1,0.077625,0.483628
2,0.093296,0.329246
3,0.083525,0.579087
4,0.439985,0.680988
...,...,...
2593664,0.012299,0.329246
2593665,0.071902,0.507993
2593666,0.190918,0.727206
2593667,0.272731,0.648089


In [36]:
# 过采样
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

logging.info('y_train distribution is %s' % Counter(y_train))
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
logging.info('y_resampled distribution is %s' % Counter(y_resampled))

# del X_train_all_v5, y_train_all

[2019-12-15 22:06:11,383] INFO: y_train distribution is Counter({0.0: 2178656, 1.0: 415013})
[2019-12-15 22:06:13,548] INFO: y_resampled distribution is Counter({0.0: 2178656, 1.0: 2178656})


In [37]:
# 手动shuffle
X_resampled = X_resampled.values
y_resampled = y_resampled.values
np.random.seed(100)
np.random.shuffle(X_resampled)
np.random.seed(100)
np.random.shuffle(y_resampled)

In [42]:
# model
model = Sequential()

model.add(Dense(256, activation='tanh', input_shape=(X_resampled.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='tanh'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='linear'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# model.summary()

In [44]:
H = model.fit(X_resampled, y_resampled, 
                epochs=50, batch_size=BATCH_SIZE, validation_split=0.1,
                shuffle='batch', callbacks=callbacks, verbose=1)

Train on 3921580 samples, validate on 435732 samples
Epoch 1/50
3921580/3921580 [==============================] - 5s 1us/step - loss: 0.4529 - accuracy: 0.7882 - val_loss: 0.4492 - val_accuracy: 0.7912
Epoch 2/50
3921580/3921580 [==============================] - 5s 1us/step - loss: 0.4522 - accuracy: 0.7886 - val_loss: 0.4473 - val_accuracy: 0.7914
Epoch 3/50
3921580/3921580 [==============================] - 5s 1us/step - loss: 0.4517 - accuracy: 0.7886 - val_loss: 0.4471 - val_accuracy: 0.7906
Epoch 4/50
3921580/3921580 [==============================] - 5s 1us/step - loss: 0.4514 - accuracy: 0.7888 - val_loss: 0.4486 - val_accuracy: 0.7903
Epoch 5/50
2023424/3921580 [==============>...............] - ETA: 2s - loss: 0.4507 - accuracy: 0.7892

KeyboardInterrupt: 

In [40]:
gdump(model, 'model_ensemble_v6')

In [117]:
model = gload('model_ensemble_v6')

In [ ]:
del X_resampled, y_resampled, X_train, y_train

# Result

## load data

In [87]:
del X_train_all, y_train_all, X_val, y_val

In [41]:
# load data
test = read_param('test')
sub = read_param('sub')
feature_cols = gload('feature_cols')
feature_cols_v5 = gload('feature_cols_v5')
test_v5 = gload('test_v5')

## predict 231

In [43]:
feature_cols = gload('feature_cols')
feature_cols_v5 = gload('feature_cols_v5')
feature_cols.extend(feature_cols_v5)
feature_cols.append('colv5')

In [44]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
x = test_v5.values
x = mms.fit_transform(x)
test_v5_tmp = pd.DataFrame(x)
test_v5_tmp.columns = feature_cols_v5
del x
test_v5_tmp[:5]

,a,b,c,d,e,aa,bb,cc,dd,ee
0,1.000000,1.000000,1.0,0.168317,0.043590,0.0,0.0,0.0,0.000000,0.000000
1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
2,1.000000,1.000000,1.0,0.217822,0.042308,1.0,1.0,1.0,0.021577,0.000918
3,0.916667,0.916667,1.0,0.311881,0.106275,0.0,0.0,0.0,0.000000,0.000000
4,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000


In [21]:
model_colv5 = gload('model_colv5')
colv5 = model_colv5.predict(test_v5_tmp, batch_size=8192)
colv5 = pd.DataFrame(colv5)
colv5.columns = ['colv5']
del test_v5_tmp
colv5

,colv5
0,0.435020
1,0.329246
2,0.451440
3,0.129826
4,0.329246
...,...
1141678,0.467790
1141679,0.455709
1141680,0.579594
1141681,0.532931


In [46]:
test = pd.concat([test, test_v5, colv5], axis=1)
test

,day,freq,gender,hour,label,q_ans_count,q_diff_qa_days_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_min,...,b,c,d,e,aa,bb,cc,dd,ee,colv5
0,3870,5,3,9,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,1.000000,34,0.453333,0,0.0,0.0,0,0.000000,0.435020
1,3872,1,3,22,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0,0.000000,0,0.0,0.0,0,0.000000,0.329246
2,3874,5,1,15,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,1.000000,44,0.440000,12,1.0,1.0,107,0.303116,0.451440
3,3873,4,2,4,NaN,4.0,4.0,11.0,3.0,0.0,...,0.916667,1.000000,63,1.105263,0,0.0,0.0,0,0.000000,0.129826
4,3872,5,1,19,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0,0.000000,0,0.0,0.0,0,0.000000,0.329246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141678,3869,5,3,20,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,0.833333,22,0.209524,0,0.0,0.0,0,0.000000,0.467790
1141679,3872,2,3,21,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,0.916667,46,0.353846,0,0.0,0.0,0,0.000000,0.455709
1141680,3871,1,3,15,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,0.727273,11,0.379310,0,0.0,0.0,0,0.000000,0.579594
1141681,3871,5,3,8,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,0.833333,37,0.077895,12,1.0,1.0,80,0.022353,0.532931


In [47]:
model_lgb_231 = gload('model_lgb_231')
sub['label'] = model_lgb_231.predict_proba(test[feature_cols])[:, 1]
sub['label']

0          0.030132
1          0.001564
2          0.059012
3          0.241707
4          0.113004
             ...   
1141678    0.114258
1141679    0.491416
1141680    0.244009
1141681    0.054261
1141682    0.082841
Name: label, Length: 1141683, dtype: float64

In [48]:
np.mean(sub['label'])

0.15444478740997264

In [49]:
sub.to_csv('./storage/result.txt', index=None, header=None, sep='\t')

## predict v4

In [119]:
final_test = pd.DataFrame()
model_lgb = gload('model_lgb')
final_test['v4'] = model_lgb.predict_proba(test[feature_cols])[:, 1]
final_test['v4']

0          0.039237
1          0.001455
2          0.158848
3          0.399821
4          0.157442
             ...   
1141678    0.114890
1141679    0.485331
1141680    0.123951
1141681    0.056573
1141682    0.079274
Name: v4, Length: 1141683, dtype: float64

## predict colv5

In [10]:
test_v5 = test_v5[feature_cols_v5]

In [94]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
x = test_v5.values
x = mms.fit_transform(x)
test_v5 = pd.DataFrame(x)
del x
test_v5[:5]

,0,1,2,3,4,5,6,7,8,9
0,1.000000,1.000000,1.0,0.168317,0.043590,0.0,0.0,0.0,0.000000,0.000000
1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
2,1.000000,1.000000,1.0,0.217822,0.042308,1.0,1.0,1.0,0.021577,0.000918
3,0.916667,0.916667,1.0,0.311881,0.106275,0.0,0.0,0.0,0.000000,0.000000
4,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000


In [96]:
model = gload('model_nn')
colv5 = model.predict(test_v5.values, batch_size=8192)
colv5 = pd.DataFrame(colv5)
colv5.columns = ['colv5']
colv5

,colv5
0,0.425989
1,0.332870
2,0.454106
3,0.110004
4,0.332870
...,...
1141678,0.469417
1141679,0.467632
1141680,0.612332
1141681,0.543289


In [97]:
# # 拼接数据
# test = pd.concat([test, colv5], axis=1)
# feature_cols.append('colv5')
# del test_v5

In [120]:
final_test['v5'] = colv5['colv5']

## output result

In [121]:
final_test

,v4,v5
0,0.039237,0.332870
1,0.001455,0.481971
2,0.158848,0.332870
3,0.399821,0.587821
4,0.157442,0.689065
...,...,...
1141678,0.114890,0.332870
1141679,0.485331,0.544487
1141680,0.123951,0.468212
1141681,0.056573,0.332870


In [123]:
model_5 = gload('model_5')
sub['label'] = model_5.predict(final_test, batch_size=8192)
sub['label']

0          0.062062
1          0.006476
2          0.282007
3          0.825728
4          0.506535
             ...   
1141678    0.204403
1141679    0.880721
1141680    0.350748
1141681    0.095675
1141682    0.072380
Name: label, Length: 1141683, dtype: float32

In [124]:
sub.to_csv('./storage/result.txt', index=None, header=None, sep='\t')

In [125]:
np.mean(sub['label'])

0.3678238

In [126]:
np.mean(final_test['v4'])

0.18850230499971676

In [17]:
# 压缩存储
for i in ['X_train_all', 'test', 'sub', 'feature_cols', 'data']:
    with read_param(i) as tmp:
        gdump(tmp, i)
    print(i)

AttributeError: __enter__